In [ ]:
# Libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping

# --- Load Data ---

df = pd.read_csv("/kaggle/input/firedetectiondataset/cleaned_dataset.csv")

# Features and target

X = df[["Temperature", "Humidity", "CO2", "Hydrogen", "Pressure"]]
y = df["Fire_Alarm"]

# --- Scale Features ---

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Split Data ---

X_train, X_test, y_train, y_test = train_test_split(
X_scaled, y, test_size=0.2, random_state=42
)

# --- Balance Data with SMOTE ---

sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

# --- Early Stopping ---

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# --- Build Model ---

model = tf.keras.Sequential([
tf.keras.layers.Dense(32, activation='relu', input_shape=(5,)),
tf.keras.layers.Dense(16, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy']
)

model.summary()

# --- Train Model ---

history = model.fit(
X_train_bal, y_train_bal,
epochs=80,
batch_size=32,
validation_split=0.2,
verbose=1,
callbacks=[early_stop],
)

# --- Evaluate Model ---

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

# --- Predict with Threshold Tuning ---

y_pred_prob = model.predict(X_test)
threshold = 0.45  # tune threshold to improve class 0 recall
y_pred = (y_pred_prob > threshold).astype(int)

# --- Classification Report ---

print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test, y_pred))

# --- Confusion Matrix ---

cm = confusion_matrix(y_test, y_pred)
print("\n=== CONFUSION MATRIX ===")
print(cm)

# --- Plot Confusion Matrix ---

plt.figure(figsize=(5,4))
plt.imshow(cm, cmap='Blues')
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks([0,1], ["No Fire", "Fire"])
plt.yticks([0,1], ["No Fire", "Fire"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# --- Save Keras Model ---

print("=== SAVING KERAS MODEL ===")
model.save("fire_model.h5")

# --- Convert to TFLite ---

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("fire_model.tflite", "wb") as f:
    f.write(tflite_model)

# --- Convert to INT8 TFLite for MCU ---

def representative_data():
    for i in np.random.choice(len(X_train), size=200, replace=False):
        yield [X_train[i:i+1].astype("float32")]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()
with open("fire_model_int8.tflite", "wb") as f:
    f.write(tflite_quant_model)

print("=== MODEL AND TFLITE FILES SAVED ===")
